Scraping from SQLBits

In [279]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import json
import pyodbc
import unicodedata

Connect to a SQLServer Database

In [280]:
server = 'sqlgla.database.windows.net'
database = 'sqlgla'
username = 'sqlgla'
password = 'Password1234!'
driver= '{SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
cursor2 = cnxn.cursor()

Connect to the SQLBits website. 

In [281]:
my_url = 'https://sqlbits.com/information/publicsessions' 
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

In [282]:
page_soup = soup(page_html, 'html.parser')

In [283]:
bitsSessions = page_soup.findAll("div", attrs={"class":"panel"})

Find the speakers name:

In [284]:
speaker = bitsSessions[0].findAll("div", attrs={"class":"col-sm-3"})[0].a['href'].replace("../speakers/","").replace("_"," ").strip()
speakerLink = bitsSessions[0].findAll("div", attrs={"class":"col-sm-3"})[0].a['href'].replace("..","https://sqlbits.com").strip()
print(speakerLink)

https://sqlbits.com/speakers/Rich_Benner


Find the session topic and level

In [285]:
topic = bitsSessions[0].findAll("div", attrs={"class":"col-sm-3"})[1].text.strip()
print(topic)

Find the session abstract:

In [286]:
abstract = bitsSessions[0].findAll("div", attrs={"class":"panel-body"})[0].text.strip()
print(abstract)

Remember those “choose your own adventure” books from when we were little? Yep, that’s what we’re doing in this session.


Find the title of the session:

In [287]:
title = bitsSessions[0].findAll("div", attrs={"class":"col-sm-6"})[0].text.strip()
print(title)

Choose your own adventure – performance edition


In [288]:
link = bitsSessions[0].findAll("div", attrs={"class":"col-sm-6"})[0].a['href'].replace("..","https://sqlbits.com").strip()
print(link)

https://sqlbits.com/sessions/event20/Choose_your_own_adventure__performance_edition


In [289]:
uClient = uReq(link)
page_html = uClient.read()
uClient.close()
abstractHTML = soup(page_html, 'html.parser')

In [290]:
abstractText = abstractHTML.findAll("div", attrs={"class":"row"})

In [291]:
abstractTagList = abstractHTML.findAll("div", attrs={"id":"TagList"})

In [292]:
abstractTags = abstractTagList[0].findAll("a")

In [293]:
abstractTags

[<a href="../../Content/?tags=Optimising" id="ctl00_ctl00_PageBody_Body_TagListView_ctrl0_TagLink">Optimising</a>,
 <a href="../../Content/?tags=On-prem" id="ctl00_ctl00_PageBody_Body_TagListView_ctrl2_TagLink">On-prem</a>,
 <a href="../../Content/?tags=Database+engine" id="ctl00_ctl00_PageBody_Body_TagListView_ctrl4_TagLink">Database engine</a>,
 <a href="../../Content/?tags=SQL+2019" id="ctl00_ctl00_PageBody_Body_TagListView_ctrl6_TagLink">SQL 2019</a>]

In [294]:
link

'https://sqlbits.com/sessions/event20/Choose_your_own_adventure__performance_edition'

In [295]:
import string 
def cleanString(strr):
    strr = unicodedata.normalize("NFKD", strr)
    strr = strr.replace('\r\n',' ')
    strr = strr.replace('\n',' ')
    strr = strr.replace('\r',' ')
    strr = strr.replace('\t',' ')
    strr = strr.replace('             ',' ')
    strr = strr.replace('  ','')
    strr = strr.replace('’','')
    strr = strr.replace('Well be voting to see where we go at each step and see what happens',' ')
           
    strr = strr.replace('                              ',' ')
    ''
    strr = strr.translate(str.maketrans('', '', string.punctuation))
    return strr

In [296]:
abstractFull = abstractText[0].findAll("div", attrs={"class":"row"})[0].text.strip()
cleanString(abstractFull)

'Were going to start with a poorly performing query and choose which route were going to take to see if we can make it better You dont need any previous experience in performance tuning for this session well briefly cover each topic as its chosenPotential topics include– Indexes– Query Design– Settings that might affect performance– Scalar Functions– SARGability– Temp Tables '

In [297]:
for tag in abstractTags:
    tagg = tag.text.strip() 
    print(cleanString(tagg))

Optimising
Onprem
Database engine
SQL 2019


In [298]:
abstractTags[0]

<a href="../../Content/?tags=Optimising" id="ctl00_ctl00_PageBody_Body_TagListView_ctrl0_TagLink">Optimising</a>

Well that does sound interesting William!

But lets go a bit deeper. Lets look at all the sessions which went in to SQLBits in 2018.

In python we can use len to find the length of a list. 

In [299]:
y = len(bitsSessions)


In [300]:
speakerLink = 'https://sqlbits.com/speakers/Simon_Whiteley'
uClient = uReq(speakerLink)
page_html = uClient.read()
uClient.close()
speakerHTML = soup(page_html, 'html.parser')

In [301]:
sessionList = speakerHTML.findAll("div", attrs={"class":"row"})[2].findAll("div", attrs={"class":"col-sm-12"})
output = []
for sl in sessionList:
    if sl.text.strip() not in output:
        output.append(sl.text.strip())
len(output)

4

In [302]:
 a[4]

<div class="col-sm-12">
<a href="../Sessions/Event20/Python_Pipeline_Primer_Data_Engineering_with_DataBricks" id="ctl00_ctl00_PageBody_Body_PendingSpeakerSessionList_ContentListView_ctrl4_SessionTitle">Python Pipeline Primer: Data Engineering with DataBricks</a>
</div>

In [303]:
len()

TypeError: len() takes exactly one argument (0 given)

In [304]:
len(speakerHTML.findAll("div", attrs={"class":"row"})[3].findAll("div", attrs={"class":"item"}))

4

In [305]:
for x in range(0, y):
    title = bitsSessions[x].findAll("div", attrs={"class":"col-sm-6"})[0].text.strip()
    abstract = bitsSessions[x].findAll("div", attrs={"class":"panel-body"})[0].text.strip()
    speaker = bitsSessions[x].findAll("div", attrs={"class":"col-sm-3"})[0].a['href'].replace("../speakers/","").replace("_"," ").strip()
    speakerLink = bitsSessions[x].findAll("div", attrs={"class":"col-sm-3"})[0].a['href'].replace("..","https://sqlbits.com").strip()
    topic = bitsSessions[x].findAll("div", attrs={"class":"col-sm-3"})[1].text.strip()
    link = bitsSessions[x].findAll("div", attrs={"class":"col-sm-6"})[0].a['href'].replace("..","https://sqlbits.com").strip()
    uClient = uReq(link)
    page_html = uClient.read()
    uClient.close()
    abstractHTML = soup(page_html, 'html.parser')
    abstractText = abstractHTML.findAll("div", attrs={"class":"row"})
    abstractTagList = abstractHTML.findAll("div", attrs={"id":"TagList"})
    abstractTags = abstractTagList[0].findAll("a")
    abstractFull = abstractText[0].findAll("div", attrs={"class":"row"})[0].text.strip()
    uClient = uReq(speakerLink)
    page_html = uClient.read()
    uClient.close()
    speakerHTML = soup(page_html, 'html.parser')
    sessionList = speakerHTML.findAll("div", attrs={"class":"row"})[2].findAll("div", attrs={"class":"col-sm-12"})
    output = []
    for sl in sessionList:
        if sl.text.strip() not in output:
            output.append(sl.text.strip())
    sub = len(output)
    sel = len(speakerHTML.findAll("div", attrs={"class":"row"})[3].findAll("div", attrs={"class":"item"}))
    print(y - x)
    
    cursor = cnxn.cursor()
    cursor.execute('INSERT INTO dbo.sqlbitssessions2 (SessionId, Title, Speaker, Topic, Abstract, AbstractFull,NumberOfSessionsSubmitted, NumberOfSessionsSelected) VALUES (?,?,?,?,?,?,?,?)',(x,title,speaker,topic,abstract, abstractFull, sub, sel))
    cnxn.commit()
    for tag in abstractTags:
        tagg = cleanString(tag.text.strip())
        #print(tagg)
        cursor = cnxn.cursor()
        cursor.execute('INSERT INTO dbo.sqlbitssessionstags (SessionId, Tag) VALUES (?,?)',(x,tagg))
        cnxn.commit()


    

573
572
571
570
569
568
567
566
565
564
563
562
561
560
559
558
557
556
555
554
553
552
551
550
549
548
547
546
545
544
543
542
541
540
539
538
537
536
535
534
533
532
531
530
529
528
527
526
525
524
523
522
521
520
519
518
517
516
515
514
513
512
511
510
509
508
507
506
505
504
503
502
501
500
499
498
497
496
495
494
493
492
491
490
489
488
487
486
485
484
483
482
481
480
479
478
477
476
475
474
473
472
471
470
469
468
467
466
465
464
463
462
461
460
459
458
457
456
455
454
453
452
451
450
449
448
447
446
445
444
443
442
441
440
439
438
437
436
435
434
433
432
431
430
429
428
427
426
425
424
423
422
421
420
419
418
417
416
415
414
413
412
411
410
409
408
407
406
405
404
403
402
401
400
399
398
397
396
395
394
393
392
391
390
389
388
387
386
385
384
383
382
381
380
379
378
377
376
375
374
373
372
371
370
369
368
367
366
365
364
363
362
361
360
359
358
357
356
355
354
353
352
351
350
349
348
347
346
345
344
343
342
341
340
339
338
337
336
335
334
333
332
331
330
329
328
327
326
325
324
